In [1]:
!pip install tensorflow


## RNN

### Predict sentiment for movie review

In [2]:

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the IMDb dataset
vocab_size = 10000
max_len = 200
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Build RNN model
model_rnn = tf.keras.Sequential([
    # Embedding layer maps word indices to dense vectors: Input Layer
    tf.keras.layers.Embedding(vocab_size, 32, input_length=max_len),

    # Simple RNN layer processes the sequence data: Hidden Layer
    tf.keras.layers.SimpleRNN(32),

    # Output layer with sigmoid for binary classification (positive/negative) : Output Layer
    tf.keras.layers.Dense(1, activation='sigmoid')
])


# Compile the model with binary crossentropy and Adam optimizer
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model_rnn.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

# Evaluate and predict
loss, acc = model_rnn.evaluate(x_test, y_test)
print("Test Accuracy (RNN):", acc)

# Predict the first review
print("True Sentiment:", y_test[:1])
print("Predicted Sentiment:", model_rnn.predict(x_test[:1]))


Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 66ms/step - accuracy: 0.5480 - loss: 0.6791 - val_accuracy: 0.7402 - val_loss: 0.5222
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 56ms/step - accuracy: 0.8254 - loss: 0.4067 - val_accuracy: 0.7436 - val_loss: 0.5191
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step - accuracy: 0.8973 - loss: 0.2625 - val_accuracy: 0.8224 - val_loss: 0.4334
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.8146 - loss: 0.4422
Test Accuracy (RNN): 0.8176800012588501
True Sentiment: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
Predicted Sentiment: [[0.11884864]]


In [3]:
print("True Sentiment:", y_test[1:2])
print("Predicted Sentiment:", model_rnn.predict(x_test[1:2]))

True Sentiment: [1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Predicted Sentiment: [[0.991928]]


In [10]:
model_rnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 966,341 (3.69 MB)

 Trainable params: 322,113 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 644,228 (2.46 MB)

## CNN

### predicting the hand written digits from images

In [4]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST digit dataset (images are 28x28 grayscale)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and normalize input images to range [0, 1]
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# One-hot encode the labels (e.g., 3 -> [0,0,0,1,0,0,0,0,0,0])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build CNN model
model_cnn = tf.keras.Sequential([
    # First convolutional layer: 32 filters, 3x3 kernel, ReLU activation
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),

    # MaxPooling to reduce spatial dimensions (downsampling)
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Flatten the 2D feature maps into 1D for the Dense layer
    tf.keras.layers.Flatten(),

    # Dense hidden layer with 64 neurons
    tf.keras.layers.Dense(64, activation='relu'),

    # Output layer: 10 neurons (for 10 digit classes), softmax activation
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile with categorical crossentropy (for multi-class) and Adam optimizer
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model_cnn.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.1)

# Evaluate and predict
loss, acc = model_cnn.evaluate(x_test, y_test)
print("Test Accuracy (CNN):", acc)

# Predict the first test image
import numpy as np
prediction = model_cnn.predict(x_test[:1])
print("Predicted Digit:", np.argmax(prediction))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 31s 32ms/step - accuracy: 0.8730 - loss: 0.4287 - val_accuracy: 0.9803 - val_loss: 0.0804
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - accuracy: 0.9759 - loss: 0.0798 - val_accuracy: 0.9857 - val_loss: 0.0623
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 41s 29ms/step - accuracy: 0.9852 - loss: 0.0519 - val_accuracy: 0.9852 - val_loss: 0.0602
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9760 - loss: 0.0697
Test Accuracy (CNN): 0.9810000061988831
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Predicted Digit: 7


In [6]:
for i , num in enumerate(prediction[:1][0]):
  print(i, num)

0 3.2444083e-07
1 4.49261e-08
2 1.1445777e-06
3 1.2528914e-05
4 6.924853e-10
5 2.0995432e-09
6 4.46145e-12
7 0.9999832
8 4.7566846e-07
9 2.3735931e-06


In [7]:
model_cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │       346,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,041,440 (3.97 MB)

 Trainable params: 347,146 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 694,294 (2.65 MB)